In [1]:
from google.cloud import bigquery as bq
import os
import pandas as pd
import pandas_gbq as pgbq
import util_functions as util
import matplotlib.pyplot as plt

%matplotlib inline

if os.environ['COMPUTERNAME'] == 'DESKTOP-9KG2MPJ':
    service_credentials = 'Service_Credentials/Big Query Horse Play-f8b9319b2129.json'
else:
    service_credentials = 'Service_Credentials/big-query-horse-play-f37757d450b8.json'

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = service_credentials

ga_data_client = bq.Client(project='bigquery-public-data')

In [ ]:
ga_dataset_ref = util.get_dataset(ga_data_client, 'google_analytics_sample')

In [7]:
query = (
"""
SELECT
    #visitId,
    device.deviceCategory,
    SUM(totals.visits) AS Visits,
    SUM(totals.pageviews) AS PVs,
    SUM(totals.newVisits) AS NewVisits
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_20170101`
GROUP BY
    #visitId,
    device.deviceCategory
""")

In [8]:
df = pgbq.read_gbq(query, dialect='standard')

In [9]:
df

,deviceCategory,Visits,PVs,NewVisits
0,desktop,759,3246,576
1,mobile,503,1678,390
2,tablet,102,438,76


In [15]:
# Testing partitioned tables and table suffix
query_mult_days = (
"""
SELECT
    date,
    device.deviceCategory,
    SUM(totals.visits) AS Visits,
    SUM(totals.pageviews) AS PVs,
    SUM(totals.newVisits) AS NewVisits
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*`
WHERE
    _TABLE_SUFFIX BETWEEN '20170101'
    AND
    '20170105'
GROUP BY
    date,
    device.deviceCategory
ORDER BY date ASC
""")

In [16]:
mult_day_df = pgbq.read_gbq(query_mult_days, dialect='standard')

In [17]:
mult_day_df

,date,deviceCategory,Visits,PVs,NewVisits
0,20170101,desktop,759,3246,576
1,20170101,mobile,503,1678,390
2,20170101,tablet,102,438,76
3,20170102,tablet,95,302,72
4,20170102,mobile,477,1794,371
5,20170102,desktop,1048,4187,784
6,20170103,tablet,70,319,47
7,20170103,mobile,568,1900,400
8,20170103,desktop,1765,7240,1018
9,20170104,mobile,566,2344,448


In [ ]:
weather_query = (
"""
WITH WashingtonStations AS (
  SELECT weather.stn AS station_id, ANY_VALUE(station.name) AS name
  FROM `bigquery-public-data.noaa_gsod.stations` AS station
  INNER JOIN `bigquery-public-data.noaa_gsod.gsod2015` AS weather
  ON station.usaf = weather.stn
  WHERE station.state = 'WA' AND station.usaf != '999999'
  GROUP BY station_id
)
SELECT washington_stations.name,
  (SELECT COUNT(*)
   FROM `bigquery-public-data.noaa_gsod.gsod2015` AS weather
   WHERE washington_stations.station_id = weather.stn
   AND max >= 70) AS warm_days
FROM WashingtonStations AS washington_stations
ORDER BY warm_days DESC;
""")

wash_weather = pgbq.read_gbq(weather_query, dialect='standard')

In [ ]:
wash_weather